In [1]:
import torch
from torch.nn import MSELoss
from tqdm import trange
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np
import pickle
import wandb

from torch.utils.data import Dataset, DataLoader, Subset, random_split
import tonic

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

class CCMKDataset(Dataset):
    def __init__(self, spikes_tensor, labels_tensor, nchannels=None, target_label=2, seed=None, device=None):
        if seed is not None:
            np.random.seed(seed)
            torch.manual_seed(seed)
        
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.spikes_tensor = spikes_tensor.to(device)
        self.labels_tensor = labels_tensor.to(device)

        if nchannels is not None:
            self.spikes_tensor = self.spikes_tensor[:, nchannels, :]
        
        # Ignore labels 1, 3, and 4
        valid_mask = (self.labels_tensor == 0) | (self.labels_tensor == target_label)
        self.spikes_tensor = self.spikes_tensor[valid_mask]
        self.labels_tensor = self.labels_tensor[valid_mask]
        
        # Convert the target label (2) to 1
        self.labels_tensor[self.labels_tensor == target_label] = 1
        
        # Downsampling: Match the number of label 0 samples to the number of label 1 samples
        label_0_indices = torch.where(self.labels_tensor == 0)[0]
        label_1_indices = torch.where(self.labels_tensor == 1)[0]
        num_samples = min(len(label_1_indices), len(label_0_indices))

        # Randomly sample from label 0 and label 1 indices
        #CPU
        # selected_label_0_indices = torch.tensor(np.random.choice(label_0_indices.cpu(), size=num_samples, replace=False), device=self.device)
        # selected_label_1_indices = torch.tensor(np.random.choice(label_1_indices.cpu(), size=num_samples, replace=False), device=self.device)

        # GPU
        selected_label_0_indices = label_0_indices[torch.randperm(len(label_0_indices))[:num_samples]].to(self.device)
        selected_label_1_indices = label_1_indices[torch.randperm(len(label_1_indices))[:num_samples]].to(self.device)


        # Combine the downsampled label 0 indices with label 1 indices
        balanced_indices = torch.cat([selected_label_0_indices, selected_label_1_indices])

        # Apply the balanced indices to spikes and labels
        self.spikes_tensor = self.spikes_tensor[balanced_indices]
        self.labels_tensor = self.labels_tensor[balanced_indices]

        # Debugging: Check the distribution of labels after processing
        print(f"Filtered Labels distribution after processing: {torch.bincount(self.labels_tensor.int().cpu())}")

    def __len__(self):
        return len(self.labels_tensor)
    
    def __getitem__(self, idx):
        spike_data = self.spikes_tensor[idx]
        label = self.labels_tensor[idx]
        return spike_data, label

c:\Users\Angela\anaconda3\envs\capstone_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_loader = torch.load('saved_loaders_bs32/train_loader.pth')
val_loader = torch.load('saved_loaders_bs32/val_loader.pth')
test_loader = torch.load('saved_loaders_bs32/test_loader.pth')

C:\Users\Angela\AppData\Local\Temp\ipykernel_93188\3645305366.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loader = torch.load('saved_loaders_bs32/train_loader.

In [3]:
from rockpool.nn.networks import SynNet
from rockpool.nn.modules import LIFTorch as LIFOtherSpiking

n_channels = 16  
n_classes = 1   
n_time = 101
dt = 10e-3     

model = SynNet(
    output="vmem",
    n_channels=n_channels,                        
    n_classes=n_classes,
    dt=dt,                           
    size_hidden_layers=[48, 24, 48],        
    time_constants_per_layer=[8, 10, 12],
    tau_syn_base=0.02,  
    tau_mem=0.02,
    tau_syn_out=0.02
)

print(model)

c:\Users\Angela\anaconda3\envs\capstone_gpu\lib\site-packages\rockpool\nn\networks\__init__.py:15: UserWarning: This module needs to be ported to teh v2 API.
  warnings.warn(f"{err}")
c:\Users\Angela\anaconda3\envs\capstone_gpu\lib\site-packages\rockpool\nn\networks\__init__.py:20: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")


SynNet  with shape (16, 1) {
    TorchSequential 'seq' with shape (16, 1) {
        LinearTorch '0_LinearTorch' with shape (16, 48)
        LIFTorch '1_LIFTorch' with shape (48, 48)
        TimeStepDropout '2_TimeStepDropout' with shape (48,)
        LinearTorch '3_LinearTorch' with shape (48, 24)
        LIFTorch '4_LIFTorch' with shape (24, 24)
        TimeStepDropout '5_TimeStepDropout' with shape (24,)
        LinearTorch '6_LinearTorch' with shape (24, 48)
        LIFTorch '7_LIFTorch' with shape (48, 48)
        TimeStepDropout '8_TimeStepDropout' with shape (48,)
        LinearTorch '9_LinearTorch' with shape (48, 1)
        LIFTorch '10_LIFTorch' with shape (1, 1)
    }
}


In [4]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "drawn_wood_training_comparing_potentials.ipynb"

In [6]:
import pickle
import numpy as np
from sklearn.metrics import precision_score, recall_score
import wandb
from torch.nn import MSELoss
from torch.optim import Adam
from tqdm import trange
import matplotlib.pyplot as plt
from rockpool.nn.networks import SynNet
from rockpool.nn.modules import LIFTorch as LIFOtherSpiking


seed = 42
learning_rate = 1e-3
size_hidden_layers=[48, 24, 48]        
time_constants_per_layer=[8, 10, 12]

device = torch.device("cpu")  

# Target class potential configurations
potential_configs = [
    {"target_class_potential": 1, "non_target_class_potential": -1, "model_name": "drawn-wood-42-1"},
    {"target_class_potential": 3, "non_target_class_potential": -3, "model_name": "drawn-wood-42-3"}
]

for potential_config in potential_configs:
    target_class_potential = potential_config["target_class_potential"]
    non_target_class_potential = potential_config["non_target_class_potential"]
    model_name = potential_config["model_name"]
    
    wandb.init(
        project="drawn_wood_42_training",  
        config={
            "learning_rate": learning_rate,
            "size_hidden_layers": size_hidden_layers,
            "time_constants_per_layer": time_constants_per_layer,
            "architecture": "SNN",
            "dataset": "Meerkat calls",
            "epochs": 200,
            "target_class_potential": target_class_potential,
            "non_target_class_potential": non_target_class_potential,
            "packages": ["torch", "numpy", "pickle", "wandb"]
        }
    )

    config = wandb.config

    torch.manual_seed(seed)

    # Initialize model 
    model = SynNet(
        output="vmem",
        n_channels=16,                        
        n_classes=1,
        dt=10e-3,                           
        size_hidden_layers=size_hidden_layers,        
        time_constants_per_layer=time_constants_per_layer,
        tau_syn_base=0.02,  
        tau_mem=0.02,
        tau_syn_out=0.02,
    ).to(device)

    # Hyperparameters
    n_epochs = config.epochs
    optimizer = Adam(model.parameters().astorch(), lr=learning_rate)
    loss_fun = MSELoss()

    # Store losses, internal states, and metrics
    train_losses = []
    val_losses = []
    membrane_potentials = []
    spike_outputs = []
    synaptic_currents = []

    # Training phase
    for epoch in trange(n_epochs):
        model.train()
        epoch_loss = 0
        predictions_train = []
        ground_truths_train = []
        
        for batch in train_loader:
            inputs, labels = batch
            inputs = inputs.permute(0, 2, 1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs, _, _ = model(inputs, record=True)
            outputs_vmem = outputs[:, -1, :].to(device)  # Last time step along the time axis

            # Target membrane potential values
            target_membrane_potential = torch.zeros_like(outputs_vmem).to(device)
            target_membrane_potential[labels == 1] = target_class_potential  # Target class
            target_membrane_potential[labels == 0] = non_target_class_potential  # Non-target class

            # Compute loss
            loss = loss_fun(outputs_vmem, target_membrane_potential)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            # Only collect predictions and ground truths during the final epoch
            if epoch == n_epochs - 1:
                predictions_train.append(outputs_vmem.detach().cpu().numpy())
                ground_truths_train.append(labels.cpu().numpy())

            # Access internal layer membrane potentials, spikes, synaptic currents
            for lyr_name in model.lif_names:
                lyr = model.seq[lyr_name]

                v_mem = lyr.vmem.detach().cpu().numpy()  
                spikes = lyr.spikes.detach().cpu().numpy()  
                i_syn = lyr.isyn.detach().cpu().numpy()  

                membrane_potentials.append(v_mem)
                spike_outputs.append(spikes)
                synaptic_currents.append(i_syn)

        train_losses.append(epoch_loss / len(train_loader))

        # Validation phase
        model.eval()
        val_loss = 0
        predictions_val = []
        ground_truths_val = []
        with torch.no_grad():
            for batch in val_loader:
                inputs, labels = batch
                inputs = inputs.permute(0, 2, 1).to(device)
                labels = labels.to(device)
                outputs, _, _ = model(inputs, record=True)
                outputs_vmem = outputs[:, -1, :].to(device)

                # Target membrane potential values
                target_membrane_potential = torch.zeros_like(outputs_vmem).to(device)
                target_membrane_potential[labels == 1] = target_class_potential
                target_membrane_potential[labels == 0] = non_target_class_potential

                # Compute validation loss
                loss = loss_fun(outputs_vmem, target_membrane_potential)
                val_loss += loss.item()

                # Only collect predictions and ground truths during the final epoch
                if epoch == n_epochs - 1:
                    predictions_val.append(outputs_vmem.detach().cpu().numpy())
                    ground_truths_val.append(labels.cpu().numpy())

        val_losses.append(val_loss / len(val_loader))

        # Log loss for the current epoch
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_losses[-1],
            "val_loss": val_losses[-1]
        })
        print(f"Epoch {epoch+1}/{n_epochs}, Training Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}")

    # Calculate metrics for training and validation at the final epoch
    if predictions_train and ground_truths_train:
        predicted_labels_train = np.where(np.concatenate(predictions_train) > 0, 1, 0).flatten()
        ground_truths_train_flat = np.concatenate(ground_truths_train).flatten()
        train_accuracy = (predicted_labels_train == ground_truths_train_flat).mean() * 100
        train_precision = precision_score(ground_truths_train_flat, predicted_labels_train)
        train_recall = recall_score(ground_truths_train_flat, predicted_labels_train)

    if predictions_val and ground_truths_val:
        predicted_labels_val = np.where(np.concatenate(predictions_val) > 0, 1, 0).flatten()
        ground_truths_val_flat = np.concatenate(ground_truths_val).flatten()
        val_accuracy = (predicted_labels_val == ground_truths_val_flat).mean() * 100
        val_precision = precision_score(ground_truths_val_flat, predicted_labels_val)
        val_recall = recall_score(ground_truths_val_flat, predicted_labels_val)

    # Log metrics from the final epoch to wandb
    wandb.log({
        "final_train_accuracy": train_accuracy,
        "final_train_precision": train_precision,
        "final_train_recall": train_recall,
        "final_val_accuracy": val_accuracy,
        "final_val_precision": val_precision,
        "final_val_recall": val_recall
    })

    print(f"Final Training Accuracy: {train_accuracy:.2f}%, Precision: {train_precision:.2f}, Recall: {train_recall:.2f}")
    print(f"Final Validation Accuracy: {val_accuracy:.2f}%, Precision: {val_precision:.2f}, Recall: {val_recall:.2f}")

    # Save results and model state for this configuration
    results_folder = "drawn_wood_results"
    os.makedirs(results_folder, exist_ok=True)

    output_filename = f"{results_folder}/{model_name}.pkl"
    with open(output_filename, 'wb') as f:
        pickle.dump({
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accuracy': train_accuracy,
            'train_precision': train_precision,
            'train_recall': train_recall,
            'val_accuracy': val_accuracy,
            'val_precision': val_precision,
            'val_recall': val_recall,
            'membrane_potentials': membrane_potentials,
            'spike_outputs': spike_outputs,
            'synaptic_currents': synaptic_currents,
            'predictions_train': predictions_train,
            'ground_truths_train': ground_truths_train,
            'predictions_val': predictions_val,
            'ground_truths_val': ground_truths_val
        }, f)

    # Save model state
    model_state_filename = f"{results_folder}/{model_name}.pt"
    torch.save(model.state_dict(), model_state_filename)

    # Finish the wandb run for this configuration
    wandb.finish()

  0%|          | 1/200 [00:15<52:40, 15.88s/it]

Epoch 1/200, Training Loss: 3.295859440317694, Validation Loss: 0.9302844405174255


  1%|          | 2/200 [00:31<52:48, 16.00s/it]

Epoch 2/200, Training Loss: 0.9412934847597806, Validation Loss: 0.7614762229578835


  2%|▏         | 3/200 [00:47<52:29, 15.99s/it]

...

100%|█████████▉| 199/200 [54:20<00:16, 16.06s/it]

Epoch 199/200, Training Loss: 1.871105063636348, Validation Loss: 3.8364622592926025


100%|██████████| 200/200 [54:36<00:00, 16.38s/it]

Epoch 200/200, Training Loss: 1.8308440649284508, Validation Loss: 3.2563063417162215
Final Training Accuracy: 95.39%, Precision: 0.96, Recall: 0.95
Final Validation Accuracy: 90.38%, Precision: 0.90, Recall: 0.91


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
final_train_accuracy,▁
final_train_precision,▁
final_train_recall,▁
final_val_accuracy,▁
final_val_precision,▁
final_val_recall,▁
train_loss,█▆▅▄▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▁▂▁▁▂▂▂▁▁▁▁▁▁▁
val_loss,█▄▄▄▄▂▂▁▃▁▂▂▂▂▁▁▂▃▁▂▁▁▁▂▁▂▂▁▂▁▂▁▂▁▂▁▁▁▁▂
epoch,200
final_train_accuracy,95.39474
